In [1]:
#cleaning the data

In [2]:
import re
import pandas as pd
import numpy as np
import requests
import os
import json
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
from IPython.core.display import HTML
from datetime import date, datetime

In [3]:
senators_file= "1976-2018-senate.csv"

In [4]:
# new_senate=pd.read_csv(senators_file)
new_senate = pd.read_csv(senators_file, encoding= 'unicode_escape')
df= pd.DataFrame(new_senate)
#df.head()

In [5]:
# Getting significant years
datayear = df[(df.year>=2014) & (df.year<=2018)]
# datayear

In [6]:
new_df=datayear[["year","state","state_po","candidate","party","candidatevotes","writein","totalvotes"]]
new_df.head(10)

,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,NaN,NaN,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,republican,795606,False,818090
2863,2014,Alaska,AK,NaN,NaN,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,libertarian,10512,False,282400
2866,2014,Alaska,AK,Ted Gianoutsos,nonaffiliated,5636,False,282400
2867,2014,Alaska,AK,Dan Sullivan,republican,135445,False,282400
2868,2014,Arkansas,AR,Tom Cotton,republican,478819,False,847505
2869,2014,Arkansas,AR,Mark L. Pryor,democrat,334174,False,847505
2870,2014,Arkansas,AR,Mark H. Swaney,green,16797,False,847505


In [7]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 560 entries, 2861 to 3420
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year            560 non-null    int64 
 1   state           560 non-null    object
 2   state_po        560 non-null    object
 3   candidate       468 non-null    object
 4   party           439 non-null    object
 5   candidatevotes  560 non-null    int64 
 6   writein         560 non-null    bool  
 7   totalvotes      560 non-null    int64 
dtypes: bool(1), int64(3), object(4)
memory usage: 35.5+ KB


In [8]:
new_df.columns

Index(['year', 'state', 'state_po', 'candidate', 'party', 'candidatevotes',
       'writein', 'totalvotes'],
      dtype='object')

In [9]:
#Replacing Nans, and renaming republican, democrat and liberal
new_df['candidate'] = new_df['candidate'].replace(np.NaN, "Write In")
new_df['party'] = new_df['party'].replace(np.NaN, "No Party")
new_df["party"]= new_df["party"].replace(["republican"], "Republican")
new_df["party"]= new_df["party"].replace(["democrat"], "Democrat")
new_df["party"]= new_df["party"].replace(["libertarian"], "Libertarian")
new_df.head(10)


/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write In,No Party,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,Republican,795606,False,818090
2863,2014,Alaska,AK,Write In,No Party,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,Democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,Libertarian,10512,False,282400
2866,2014,Alaska,AK,Ted Gianoutsos,nonaffiliated,5636,False,282400
2867,2014,Alaska,AK,Dan Sullivan,Republican,135445,False,282400
2868,2014,Arkansas,AR,Tom Cotton,Republican,478819,False,847505
2869,2014,Arkansas,AR,Mark L. Pryor,Democrat,334174,False,847505
2870,2014,Arkansas,AR,Mark H. Swaney,green,16797,False,847505


In [10]:
new_df.shape

(560, 8)

In [11]:
test = []
firstName = []
lastName = []
for i in new_df['candidate']:
    if i == 'Write In':
        firstName.append('NaN')
        lastName.append('NaN')
    else:
        holder = i.split()
        firstName.append(holder[0])
        lastName.append(holder[-1])
print(firstName[:10])
print(lastName[:10])
new_df['Last_Name'] = lastName
new_df['First_Name'] = firstName
new_df = new_df[['year', 'party', 'candidate', 'Last_Name', 'First_Name', 'state_po', 'candidatevotes', 'state', 'totalvotes', 'writein']]
new_df.head(10)

['NaN', 'Jeff', 'NaN', 'Mark', 'Mark', 'Ted', 'Dan', 'Tom', 'Mark', 'Mark']
['NaN', 'Sessions', 'NaN', 'Begich', 'Fish', 'Gianoutsos', 'Sullivan', 'Cotton', 'Pryor', 'Swaney']


/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,year,party,candidate,Last_Name,First_Name,state_po,candidatevotes,state,totalvotes,writein
2861,2014,No Party,Write In,NaN,NaN,AL,22484,Alabama,818090,True
2862,2014,Republican,Jeff Sessions,Sessions,Jeff,AL,795606,Alabama,818090,False
2863,2014,No Party,Write In,NaN,NaN,AK,1376,Alaska,282400,True
2864,2014,Democrat,Mark Begich,Begich,Mark,AK,129431,Alaska,282400,False
2865,2014,Libertarian,Mark S. Fish,Fish,Mark,AK,10512,Alaska,282400,False
2866,2014,nonaffiliated,Ted Gianoutsos,Gianoutsos,Ted,AK,5636,Alaska,282400,False
2867,2014,Republican,Dan Sullivan,Sullivan,Dan,AK,135445,Alaska,282400,False
2868,2014,Republican,Tom Cotton,Cotton,Tom,AR,478819,Arkansas,847505,False
2869,2014,Democrat,Mark L. Pryor,Pryor,Mark,AR,334174,Arkansas,847505,False
2870,2014,green,Mark H. Swaney,Swaney,Mark,AR,16797,Arkansas,847505,False


In [12]:
# Old code, needed to normalize the data more, split name first and last
# new_df.to_csv('votersinfo.csv', index = False)

In [13]:
def matchLists(longerList, smallerList):
    index = []
    for i in smallerList:
        #print(i)
        for j in range(len(longerList)):
            if i in longerList[j]:
                #print(j)
                index.append(j)
    #print(f'length with all: {len(index)}')
    index = sorted(index)
    #print(index)
    uniqueL = []
    for x in index:
        if x not in uniqueL:
            uniqueL.append(x)
    print(f'length uniques only: {len(uniqueL)}')
    return uniqueL

In [14]:
def whichState(states):
    elections = []
    for i in states:
        if i not in elections:
            elections.append(i)
    return elections

In [15]:
votersInfo2014 = new_df[(new_df.year == 2014)].reset_index()
votersInfo2014.head()

,index,year,party,candidate,Last_Name,First_Name,state_po,candidatevotes,state,totalvotes,writein
0,2861,2014,No Party,Write In,NaN,NaN,AL,22484,Alabama,818090,True
1,2862,2014,Republican,Jeff Sessions,Sessions,Jeff,AL,795606,Alabama,818090,False
2,2863,2014,No Party,Write In,NaN,NaN,AK,1376,Alaska,282400,True
3,2864,2014,Democrat,Mark Begich,Begich,Mark,AK,129431,Alaska,282400,False
4,2865,2014,Libertarian,Mark S. Fish,Fish,Mark,AK,10512,Alaska,282400,False


# Formatting 2014
- get year
- rename cols
- splif df into republican and democrat
- exporting csv

In [16]:
votersInfo2014 = votersInfo2014.drop(['year', 'writein', 'index'], axis = 1)
votersInfo2014 = votersInfo2014.rename(columns= {'state_po':'State_Abbrv', 'state':'State', 'candidate':'Candidate', 'party':'Party', 'candidatevotes':'Candidate_Votes', 'totalvotes':"Total_Votes"})
print(len(votersInfo2014))
#votersInfo2014[:10]

187


In [17]:
votersInfo2014.head(10)

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes
0,No Party,Write In,NaN,NaN,AL,22484,Alabama,818090
1,Republican,Jeff Sessions,Sessions,Jeff,AL,795606,Alabama,818090
2,No Party,Write In,NaN,NaN,AK,1376,Alaska,282400
3,Democrat,Mark Begich,Begich,Mark,AK,129431,Alaska,282400
4,Libertarian,Mark S. Fish,Fish,Mark,AK,10512,Alaska,282400
5,nonaffiliated,Ted Gianoutsos,Gianoutsos,Ted,AK,5636,Alaska,282400
6,Republican,Dan Sullivan,Sullivan,Dan,AK,135445,Alaska,282400
7,Republican,Tom Cotton,Cotton,Tom,AR,478819,Arkansas,847505
8,Democrat,Mark L. Pryor,Pryor,Mark,AR,334174,Arkansas,847505
9,green,Mark H. Swaney,Swaney,Mark,AR,16797,Arkansas,847505


In [18]:
#Splitting Dataframe into two major parties
smallerList = ['Repub']
longerList = votersInfo2014['Party']
print('Republicans')
index2014 = matchLists(longerList, smallerList)
repubParty2014 = votersInfo2014.iloc[index2014, :]
repubParty2014 = repubParty2014.reset_index()
repubParty2014 = repubParty2014.drop(['index'], axis = 1)
##
smallerList = ['Demo']
longerList = votersInfo2014['Party']
print('Democrats')
index2014 = matchLists(longerList, smallerList)
democParty2014 = votersInfo2014.iloc[index2014, :]
democParty2014 = democParty2014.reset_index()
democParty2014 = democParty2014.drop(['index'], axis = 1)

Republicans
length uniques only: 38
Democrats
length uniques only: 37


In [19]:
repubParty2014.head(5)

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes
0,Republican,Jeff Sessions,Sessions,Jeff,AL,795606,Alabama,818090
1,Republican,Dan Sullivan,Sullivan,Dan,AK,135445,Alaska,282400
2,Republican,Tom Cotton,Cotton,Tom,AR,478819,Arkansas,847505
3,Republican,Cory Gardner,Gardner,Cory,CO,983891,Colorado,2041058
4,Republican,Kevin Wade,Wade,Kevin,DE,98823,Delaware,234038


In [20]:
democParty2014.head(5)

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes
0,Democrat,Mark Begich,Begich,Mark,AK,129431,Alaska,282400
1,Democrat,Mark L. Pryor,Pryor,Mark,AR,334174,Arkansas,847505
2,Democrat,Mark Udall,Udall,Mark,CO,944203,Colorado,2041058
3,Democrat,Christopher A. Coons,Coons,Christopher,DE,130655,Delaware,234038
4,Democrat,M. Michelle Nunn,Nunn,M.,GA,1160811,Georgia,2567805


In [21]:
repubParty2014.to_csv('repubParty2014.csv', index = False)
democParty2014.to_csv('democParty2014.csv', index = False)

In [22]:
# smallerList = ['Demo']
# longerList = votersInfo2014['Party']
# index2014 = matchLists(longerList, smallerList)
# democParty2014 = votersInfo2014.iloc[index2014, :]
# democParty2014 = democParty2014.reset_index()
# democParty2014 = democParty2014.drop(['index'], axis = 1)
# democParty2014.head(5)

In [23]:
#which states elected in 2014
stateList = list(votersInfo2014['State'])
election2014 = whichState(votersInfo2014['State'])
print(election2014)
len(election2014)

['Alabama', 'Alaska', 'Arkansas', 'Colorado', 'Delaware', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Montana', 'Nebraska', 'New Hampshire', 'New Jersey', 'New Mexico', 'North Carolina', 'Oklahoma', 'Oregon', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Virginia', 'West Virginia', 'Wyoming']


34

# Formatting 2016
- get year
- rename cols
- splif df into republican and democrat
- exporting csv

In [24]:
votersInfo2016 = new_df[(new_df.year == 2016)].reset_index()
votersInfo2016.head()

,index,year,party,candidate,Last_Name,First_Name,state_po,candidatevotes,state,totalvotes,writein
0,3048,2016,Democrat,Ron Crumpton,Crumpton,Ron,AL,748709,Alabama,2087444,False
1,3049,2016,Republican,Richard C. Shelby,Shelby,Richard,AL,1335104,Alabama,2087444,False
2,3050,2016,No Party,Write In,NaN,NaN,AL,3631,Alabama,2087444,True
3,3051,2016,Libertarian,Joe Miller,Miller,Joe,AK,90825,Alaska,311441,False
4,3052,2016,independent,Ted Gianoutsos,Gianoutsos,Ted,AK,1758,Alaska,311441,False


In [25]:
votersInfo2016 = votersInfo2016.drop(['year', 'writein', 'index'], axis = 1)
votersInfo2016 = votersInfo2016.rename(columns= {'state_po':'State_Abbrv', 'state':'State', 'candidate':'Candidate', 'party':'Party', 'candidatevotes':'Candidate_Votes', 'totalvotes':"Total_Votes"})
print(len(votersInfo2016))
votersInfo2016[:10]

221


,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes
0,Democrat,Ron Crumpton,Crumpton,Ron,AL,748709,Alabama,2087444
1,Republican,Richard C. Shelby,Shelby,Richard,AL,1335104,Alabama,2087444
2,No Party,Write In,NaN,NaN,AL,3631,Alabama,2087444
3,Libertarian,Joe Miller,Miller,Joe,AK,90825,Alaska,311441
4,independent,Ted Gianoutsos,Gianoutsos,Ted,AK,1758,Alaska,311441
5,Democrat,Ray Metcalfe,Metcalfe,Ray,AK,36200,Alaska,311441
6,independent,Margaret Stock,Stock,Margaret,AK,41194,Alaska,311441
7,independent,Breck A. Craig,Craig,Breck,AK,2609,Alaska,311441
8,Republican,Lisa Murkowski,Murkowski,Lisa,AK,138149,Alaska,311441
9,No Party,Write In,NaN,NaN,AK,706,Alaska,311441


In [26]:
#which states elected in 2016
stateList = list(votersInfo2016['State'])
election2016 = whichState(votersInfo2016['State'])
print(election2016)
len(election2016)

['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maryland', 'Missouri', 'Nevada', 'New Hampshire', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'South Carolina', 'South Dakota', 'Utah', 'Vermont', 'Washington', 'Wisconsin']


34

In [27]:
#Splitting Dataframe into two major parties
smallerList = ['Repub']
longerList = votersInfo2016['Party']
print('Republicans')
index2016 = matchLists(longerList, smallerList)
repubParty2016 = votersInfo2016.iloc[index2016, :]
repubParty2016 = repubParty2016.reset_index()
repubParty2016 = repubParty2016.drop(['index'], axis = 1)
##
smallerList = ['Demo']
longerList = votersInfo2016['Party']
print('Democrats')
index2016 = matchLists(longerList, smallerList)
democParty2016 = votersInfo2016.iloc[index2016, :]
democParty2016 = democParty2016.reset_index()
democParty2016 = democParty2016.drop(['index'], axis = 1)

Republicans
length uniques only: 42
Democrats
length uniques only: 41


In [28]:
repubParty2016.head()

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes
0,Republican,Richard C. Shelby,Shelby,Richard,AL,1335104,Alabama,2087444
1,Republican,Lisa Murkowski,Murkowski,Lisa,AK,138149,Alaska,311441
2,Republican,Sydney Dudikoff,Dudikoff,Sydney,AZ,494,Arizona,2530730
3,Republican,John McCain,McCain,John,AZ,1359267,Arizona,2530730
4,Republican,John Boozman,Boozman,John,AR,661984,Arkansas,1107522


In [29]:
democParty2016.head()

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes
0,Democrat,Ron Crumpton,Crumpton,Ron,AL,748709,Alabama,2087444
1,Democrat,Ray Metcalfe,Metcalfe,Ray,AK,36200,Alaska,311441
2,Democrat,Ann Kirkpatrick,Kirkpatrick,Ann,AZ,1031245,Arizona,2530730
3,Democrat,Conner Eldridge,Eldridge,Conner,AR,400602,Arkansas,1107522
4,Democrat,Loretta Sanchez,Sanchez,Loretta,CA,4701417,California,12244170


In [30]:
repubParty2016.to_csv('repubParty2016.csv', index = False)
democParty2016.to_csv('democParty2016.csv', index = False)

# Formatting 2018
- get year
- rename cols
- splif df into republican and democrat
- exporting csv

In [31]:
votersInfo2018 = new_df[(new_df.year == 2018)].reset_index()
#votersInfo2018.head()

In [32]:
votersInfo2018 = votersInfo2018.drop(['year', 'writein', 'index'], axis = 1)
votersInfo2018 = votersInfo2018.rename(columns= {'state_po':'State_Abbrv', 'state':'State', 'candidate':'Candidate', 'party':'Party', 'candidatevotes':'Candidate_Votes', 'totalvotes':"Total_Votes"})
print(len(votersInfo2018))
votersInfo2018[:10]

152


,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes
0,Republican,Martha McSally,McSally,Martha,AZ,1135200,Arizona,2384308
1,Democrat,Kyrsten Sinema,Sinema,Kyrsten,AZ,1191100,Arizona,2384308
2,green,Angela Green,Green,Angela,AZ,57442,Arizona,2384308
3,No Party,Write In,NaN,NaN,AZ,566,Arizona,2384308
4,Democrat,Dianne Feinstein,Feinstein,Dianne,CA,6019422,California,11113364
5,Democrat,Kevin De Leon,Leon,Kevin,CA,5093942,California,11113364
6,Democrat,Christopher S Murphy,Murphy,Christopher,CT,787685,Connecticut,1386840
7,Republican,Matthew Corey,Corey,Matthew,CT,545717,Connecticut,1386840
8,working families,Christopher S Murphy,Murphy,Christopher,CT,37894,Connecticut,1386840
9,Libertarian,Richard Lion,Lion,Richard,CT,8838,Connecticut,1386840


In [33]:
#which states elected in 2018
stateList = list(votersInfo2018['State'])
election2018 = whichState(votersInfo2018['State'])
print(election2018)
len(election2018)

['Arizona', 'California', 'Connecticut', 'Delaware', 'Florida', 'Hawaii', 'Indiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Jersey', 'New Mexico', 'New York', 'North Dakota', 'Ohio', 'Pennsylvania', 'Rhode Island', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']


33

In [34]:
#Splitting Dataframe into two major parties
smallerList = ['Repub']
longerList = votersInfo2018['Party']
print('Republicans')
index2018 = matchLists(longerList, smallerList)
repubParty2018 = votersInfo2018.iloc[index2018, :]
repubParty2018 = repubParty2018.reset_index()
repubParty2018 = repubParty2018.drop(['index'], axis = 1)
##
smallerList = ['Demo']
longerList = votersInfo2018['Party']
print('Democrats')
index2018 = matchLists(longerList, smallerList)
democParty2018 = votersInfo2018.iloc[index2018, :]
democParty2018 = democParty2018.reset_index()
democParty2018 = democParty2018.drop(['index'], axis = 1)

Republicans
length uniques only: 34
Democrats
length uniques only: 32


In [35]:
repubParty2018.head()

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes
0,Republican,Martha McSally,McSally,Martha,AZ,1135200,Arizona,2384308
1,Republican,Matthew Corey,Corey,Matthew,CT,545717,Connecticut,1386840
2,Republican,Robert B Arlett,Arlett,Robert,DE,137127,Delaware,362592
3,Republican,Rick Scott,Scott,Rick,FL,4099505,Florida,8190005
4,Republican,Ron Curtis,Curtis,Ron,HI,112035,Hawaii,388351


In [36]:
democParty2018.head()

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes
0,Democrat,Kyrsten Sinema,Sinema,Kyrsten,AZ,1191100,Arizona,2384308
1,Democrat,Dianne Feinstein,Feinstein,Dianne,CA,6019422,California,11113364
2,Democrat,Kevin De Leon,Leon,Kevin,CA,5093942,California,11113364
3,Democrat,Christopher S Murphy,Murphy,Christopher,CT,787685,Connecticut,1386840
4,Democrat,Thomas R Carper,Carper,Thomas,DE,217385,Delaware,362592


In [37]:
repubParty2018.to_csv('repubParty2018.csv', index = False)
democParty2018.to_csv('democParty2018.csv', index = False)

# CSVs 4 & 5
- Bringing in Chair_Members to match with votersInfo
- Bringing in Ranking_Members to match with votersInfo



In [38]:
#Bringing in Chair_Members to match with votersInfo
chairs_file= "Chair_Members.csv"
matchingChairs = pd.read_csv(chairs_file, encoding= 'unicode_escape')

In [39]:
#matchingChairs.head()

In [40]:
# Dropping and renaming cols
matchingChairs = matchingChairs[(matchingChairs.Committee == "Committee")].reset_index()
matchingChairs = matchingChairs.drop(['State_Abbrv', 'State','Committee', "C_Party", "C_Party_Name", "index"], axis = 1)
matchingChairs = matchingChairs.rename(columns= {'SubCommittee':'Committee'})

In [41]:
len(matchingChairs)

16

In [42]:
#Bringing in Ranking_Members to match with votersInfo
ranking_file= "Ranking_Members.csv"
matchingRanking = pd.read_csv(ranking_file, encoding= 'unicode_escape')

In [43]:
matchingRanking = matchingRanking[(matchingRanking.Committee == "Committee")].reset_index()
matchingRanking = matchingRanking.drop(['Committee', "RM_Party", "R_Party_Name", "index"], axis = 1)
matchingRanking = matchingRanking.rename(columns= {'SubCommittee':'Committee'})

In [44]:
len(matchingRanking)

16

In [45]:
matchingRanking

,Committee,R_First_Name,R_Last_Name,State_Abbrv,Ranking_Members,State
0,"Agriculture, Nutrition and Forestry",Debbie,Stabenow,MI,Debbie Stabenow,Michigan
1,Appropriations,Patrick,Leahy,VT,Patrick Leahy,Vermont
2,Armed Services,Jack,Reed,RI,Jack Reed,Rhode Island
3,"Banking, Housing and Urban Affairs",Sherrod,Brown,OH,Sherrod Brown,Ohio
4,Budget,Bernie,Sanders,VT,Bernie Sanders,Vermont
5,"Commerce, Science and Transportation",Maria,Cantwell,WA,Maria Cantwell,Washington
6,Energy and Natural Resources,Joe,Manchin,WV,Joe Manchin,West Virginia
7,Environment and Public Works,Tom,Carper,DE,Tom Carper,Delaware
8,Finance,Ron,Wyden,OR,Ron Wyden,Oregon
9,Foreign Relations,Bob,Menendez,NJ,Bob Menendez,New Jersey


# Cleaning/Removing dups 2014
- votersinfo2018 has duplicate rows of a few names, searched for names and added their total votes
- this probably occured because of how the party was entered

In [46]:
votersInfo2014['Candidate'][38] = 'James D. Oberweis'
votersInfo2014['Candidate'][68] = 'Other'
votersInfo2014['Candidate'][164] = "Emily Sanchez"

/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipyk

In [47]:
#votersInfo2014.iloc[90:150]

# Cleaning/Removing dups 2016
- votersinfo2018 has duplicate rows of a few names, searched for names and added their total votes
- this probably occured because of how the party was entered

In [48]:
# Renaming rows to fix middle names, write ins and others
votersInfo2016['Candidate'][97] = "M. V. Mendoza"
votersInfo2016['Candidate'][95] = "Charles W. Boustany Jr."
votersInfo2016['Candidate'][102] = "Rob Maness"
votersInfo2016['Candidate'][105] = "William Robert LangJr"
votersInfo2016['Candidate'][110] = "Gregory TaylorJr"
votersInfo2016['Candidate'][115] = "Joseph Cao"
votersInfo2016['Candidate'][117] = "Donald Crawford"
votersInfo2016['Candidate'][140] = "Thomas Sawyer"
votersInfo2016['Candidate'][144] = "Other"
votersInfo2016['Candidate'][154] = "Other"
votersInfo2016['Candidate'][155] = "Other"
votersInfo2016['Candidate_Votes'][34] = 920766+87948
votersInfo2016['Candidate'][56] = "Other"
votersInfo2016['Candidate'][57] = "John M. Giuffre"
votersInfo2016['Candidate'][61] = "Other"
votersInfo2016['Candidate_Votes'][151] = 45402+4784220+150655
votersInfo2016['Candidate_Votes'][152] = 267613+1723927+17813
votersInfo2016['Candidate'][205] = "Other"


/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [49]:
#votersInfo2016.iloc[200:220]

In [50]:
#Dropping Write Ins and Others, because of later function conflict, if wanted can return
votersInfo2016 = votersInfo2016.drop(votersInfo2016.index[35])
votersInfo2016 = votersInfo2016.drop(votersInfo2016.index[156])
votersInfo2016 = votersInfo2016.drop(votersInfo2016.index[158])
votersInfo2016 = votersInfo2016.drop(votersInfo2016.index[161])
votersInfo2016 = votersInfo2016.drop(votersInfo2016.index[155])
votersInfo2016 = votersInfo2016.drop(votersInfo2016.index[159])
votersInfo2016 = votersInfo2016.reset_index()
votersInfo2016 = votersInfo2016.drop(['index'], axis = 1)

# Cleaning/Removing dups 2018
- votersinfo2018 has duplicate rows of a few names, searched for names and added their total votes
- this probably occured because of how the party was entered

In [51]:
#Found dups 2018
# Kirsten E. Gillibrand
# Chele Chiavacci Farley
# Chele Chiavacci Farley
# Kirsten E. Gillibrand
# Kirsten E. Gillibrand
# Kirsten E. Gillibrand
# Chele Chiavacci Farley
# Christopher S Murphy
# No other candidates

In [52]:
#There is multiple duplicate vote numbers for New York's three candidates
#finding those locations
# for i in range(len(votersInfo2018)):
#     if votersInfo2018['Candidate'][i] == "Kirsten E. Gillibrand":
#         print(f'Gillibrand at {i}')
#     elif votersInfo2018['Candidate'][i] == "Chele Chiavacci Farley":
#         print(f'Farley at {i}')
#     elif votersInfo2018['Candidate'][i] == "Christopher S Murphy":
#         print(f'Murphey at {i}')
    

In [53]:
#Adding up murphey, farleys, and gillibrands votes
#Cleaning DF
murphey = 37894 + 787685
votersInfo2018['Candidate_Votes'][6] = murphey
votersInfo2018 = votersInfo2018.drop(votersInfo2018.index[8])
votersInfo2018 = votersInfo2018.reset_index()
votersInfo2018 = votersInfo2018.drop(['index'], axis = 1)
#Adding up Farleys votes
farley = 1730439 + 246171 + 21610
print(farley)
#Adding up Gillibrands votes
gillibrand = 3755489+160128+99325+41989
#print(gillibrand)

1998220


/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [54]:
votersInfo2018 = votersInfo2018.drop(votersInfo2018.index[91:96])
votersInfo2018 = votersInfo2018.reset_index()
votersInfo2018 = votersInfo2018.drop(['index'], axis = 1)
votersInfo2018 = votersInfo2018.drop(votersInfo2018.index[74])
votersInfo2018 = votersInfo2018.reset_index()
votersInfo2018 = votersInfo2018.drop(['index'], axis = 1)
#votersInfo2018.iloc[88:98]
votersInfo2018['Candidate_Votes'][89] = gillibrand
votersInfo2018['Candidate_Votes'][90] = farley

/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


# Removing Special Chars
- function removes special characters

In [55]:
def noChars(stripList):
    cleanedList = []
    for longName in stripList:
        counter = 0
        noChars = ''
        for letter in longName:
            if (letter == '.') & (counter < len(longName)):
                counter += 1
                continue
            elif (letter == "'") & (counter < len(longName)):
                #print(longName[counter-1])
                #noChars += longName[counter-1]
                counter += 1
            elif (letter == '"') & (counter < len(longName)):
                counter += 1
                continue
            elif (letter == ',') & (counter < len(longName)):
                counter += 1
                continue
            elif counter <= len(longName):
                noChars += letter
                counter += 1
            else:
                counter += 1
        cleanedList.append(noChars)
    return cleanedList


In [56]:
# Testing noChars
# list2 = ['Angus S. "K"ing, Jr.', '"Bob" B. B,arker', 'Write In', 'Lily C. B. Allen III', 'Amelia Airheart', "Bob O'Smirnoff"]
# clean1 = noChars(list2)
# print(clean1)

# Splitting Name
- into first, last, middle, and suffixes if possible
- need to create a list of suffixes and honorifics, ie phds, because the last grouping of strings from a name if they're an honorific are longer than 3 chars, and some names are that long or less
- have to find a way to keep names that are 3 chars or less

In [57]:
#Testing splitname first step
# splitName2018 = []
# for i in votersInfo2018['Candidate']:
#     splitName2018.append(i.split())

In [58]:
# print(len(votersInfo2018['Candidate']))
# print(len(splitName2018))

In [59]:
# Testing SplitName Func
# splitNameList = splitName(clean1)
# lastName = splitNameList[2]
# print(lastName)
# splitNameList[4]

In [60]:
def splitName(column):
    splitName = []
    lastName = []
    index = []
    for i in column:
        splitName.append(i.split())
    #print(len(splitName))
    #print(splitName)
    firstName = []
    lastName = []
    middleI = []
    lastName = []
    suffix = []
    counter = 0
    for i in splitName:
        #print(len(i))
        if len(i) == 2 and (i[0].lower() != 'write'):
            firstName.append(i[0])
            lastName.append(i[1])
            index.append(counter)
            counter += 1
        elif len(i) ==3:
            firstName.append(i[0])
            middleI.append(i[1])
            lastName.append(i[2])
            index.append(counter)
            counter += 1
        elif len(i) == 4:
            firstName.append(i[0])
            middleI.append(i[1])
            suffix.append(i[-1])
            lastName.append(i[-2])
            index.append(counter)
            counter += 1
        elif (i[0] == 'Write') | (i[0] == 'Others') | (i[0] == 'scatter')|(i[0] == 'Blank Vote') |(i[0] == 'Over Vote')| (i[0] == 'None of these Candidates') |(i[0] == 'Other'):
            counter += 1
            continue
        else:
            lastName.append('twoMiddle')
            counter += 1
            index.append(counter)
    return firstName, middleI, lastName, suffix, index

In [61]:
splitVotersInfo2014 = splitName(votersInfo2014['Candidate'])
print(len(splitVotersInfo2014[2]))
print(len(splitVotersInfo2014[4]))
lastName2014 = splitVotersInfo2014[2]
matchList = splitVotersInfo2014[4]
print(lastName2014[:20])

148
148
['Sessions', 'Begich', 'Fish', 'Gianoutsos', 'Sullivan', 'Cotton', 'Pryor', 'Swaney', 'LaFrance', 'Acosta', 'Gardner', 'Kent', 'Hammons', 'Shogan', 'Udall', 'Wade', 'Coons', 'Groff', 'Perdue', 'Nunn']


In [62]:
splitVotersInfo2016 = splitName(votersInfo2016['Candidate'])
print(len(splitVotersInfo2016[2]))
print(len(splitVotersInfo2016[4]))
lastName2016 = splitVotersInfo2016[2]
matchList = splitVotersInfo2016[4]
print(lastName2016[:20])

163
163
['Crumpton', 'Shelby', 'Miller', 'Gianoutsos', 'Metcalfe', 'Stock', 'Craig', 'Murkowski', 'Camboni', 'Kirkpatrick', 'Clark', 'Dudikoff', 'McCain', 'Swing', 'Boozman', 'Eldridge', 'Gilbert', 'Sanchez', 'Harris', 'Fiorino']


In [63]:
splitVotersInfo2018 = splitName(votersInfo2018['Candidate'])
print(len(splitVotersInfo2018[2]))
print(len(splitVotersInfo2018[4]))
lastName2018 = splitVotersInfo2018[2]
matchList = splitVotersInfo2018[4]
print(lastName2018[:20])

126
126
['McSally', 'Sinema', 'Green', 'Feinstein', 'Leon', 'Murphy', 'Corey', 'Lion', 'Russell', 'Carper', 'Arlett', 'Theodoropoulos', 'Frost', 'Scott', 'Nelson', 'Hirono', 'Curtis', 'Braun', 'Donnelly', 'Brenton']


In [64]:
#This gets the indexes of the locations that dont have write in or others
getIndex = []
howMany = []
for i in range(len(votersInfo2014)):
    if (votersInfo2014['Candidate'][i] != 'Write in') & (votersInfo2014['Candidate'][i] != 'Others') & (votersInfo2014['Candidate'][i] != 'None of these Candidates'):
        howMany.append(votersInfo2014['Candidate'][i])
        holder = votersInfo2014['Candidate'][i]
        getIndex.append(i)
        #print(f'name : {holder}, index: {i}')
print(len(howMany))
noWriteIns2014 = votersInfo2014.iloc[getIndex, :]
print(len(noWriteIns2014))
#noWriteIns

187
187


In [65]:
#This gets the indexes of the locations that dont have write in or others
getIndex = []
howMany = []
for i in range(len(votersInfo2018)):
    if (votersInfo2018['Candidate'][i] != 'Write in') & (votersInfo2018['Candidate'][i] != 'Others') & (votersInfo2018['Candidate'][i] != 'None of these Candidates'):
        howMany.append(votersInfo2018['Candidate'][i])
        holder = votersInfo2018['Candidate'][i]
        getIndex.append(i)
        #print(f'name : {holder}, index: {i}')
print(len(howMany))
noWriteIns2018 = votersInfo2018.iloc[getIndex, :]
print(len(noWriteIns2018))
#noWriteIns

143
143


In [66]:
#This gets the indexes of the locations that dont have write in or others
getIndex = []
howMany = []
for i in range(len(votersInfo2016)):
    if (votersInfo2016['Candidate'][i] != 'Write in') & (votersInfo2016['Candidate'][i] != 'Others') & (votersInfo2016['Candidate'][i] != 'None of these Candidates'):
        howMany.append(votersInfo2016['Candidate'][i])
        holder = votersInfo2016['Candidate'][i]
        getIndex.append(i)
        #print(f'name : {holder}, index: {i}')
print(len(howMany))
noWriteIns2016 = votersInfo2016.iloc[getIndex, :]
print(len(noWriteIns2016))
#noWriteIns

215
215


In [67]:
#Keep this code!!!!!
#this matches two lists
def matchLists(longerList, smallerList):
    index = []
    for i in smallerList:
        #print(i)
        for j in range(len(longerList)):
            if i in longerList[j]:
                #print(j)
                index.append(j)
    print(f'length with all: {len(index)}')
    index = sorted(index)
    #print(index)
    uniqueL = []
    for x in index:
        if x not in uniqueL:
            uniqueL.append(x)
    print(f'length uniques only: {len(uniqueL)}')
    return uniqueL

In [68]:
#This code Looks for an exact match
# list1 is longer
# def exactNonMatches(longerList, shorterList):
#     nonMatches = [i for i in shorterList if i not in longerList]
#     return nonMatches
# smallList = ['Mary Sue', 'Parks']
# longerList = ['Lilly Anne', 'George Parks', 'Bobette Shirly', 'Mary Sue']
# print(matchLists(longerList, smallList))
#print(exactNonMatches(longerList, smallList))

In [69]:
longName2014 = list(noWriteIns2014['Candidate']) 
longName2016 = list(noWriteIns2016['Candidate']) 
longName2018 = list(noWriteIns2018['Candidate']) 

In [70]:
#Checking lists
indexes2014 = matchLists(longName2014, lastName2014)
indexes2016 = matchLists(longName2016, lastName2016)
indexes2018 = matchLists(longName2018, lastName2018)

length with all: 185
length uniques only: 148
length with all: 204
length uniques only: 163
length with all: 133
length uniques only: 126


In [71]:
#Next line drops all rows that are not in the indexes list
votersLast2014 = noWriteIns2014.iloc[indexes2014, :]
votersLast2014 = votersLast2014.reset_index()
votersLast2014 = votersLast2014.drop(['index'], axis = 1)

In [72]:
#Next line drops all rows that are not in the indexes list
votersLast2016 = noWriteIns2016.iloc[indexes2016, :]
votersLast2016 = votersLast2016.reset_index()
votersLast2016 = votersLast2016.drop(['index'], axis = 1)

In [73]:
#Next line drops all rows that are not in the indexes list
votersLast2018 = noWriteIns2018.iloc[indexes2018, :]
votersLast2018 = votersLast2018.reset_index()
votersLast2018 = votersLast2018.drop(['index'], axis = 1)

In [74]:
#lastName2018[:10]

In [75]:
lastName2014 = pd.Series(lastName2014)
#lastName2014[3]
lastName2016 = pd.Series(lastName2016)
#lastName2016[3]
lastName2018 = pd.Series(lastName2018)
#lastName2018[3]

In [76]:
lastName2016 = pd.Series(lastName2016)
# lastName2016[3]

In [77]:
lastName2018 = pd.Series(lastName2018)
# lastName2018[3]

# Isolating Last Name
- the next code adds the lastName row to the DF, and uses iloc to test if matches

In [78]:
votersLast2014.head(5)

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes
0,Republican,Jeff Sessions,Sessions,Jeff,AL,795606,Alabama,818090
1,Democrat,Mark Begich,Begich,Mark,AK,129431,Alaska,282400
2,Libertarian,Mark S. Fish,Fish,Mark,AK,10512,Alaska,282400
3,nonaffiliated,Ted Gianoutsos,Gianoutsos,Ted,AK,5636,Alaska,282400
4,Republican,Dan Sullivan,Sullivan,Dan,AK,135445,Alaska,282400


In [79]:
print(len(votersLast2014))

148


In [80]:
votersLast2014['lastName'] = lastName2014
votersLast2016['lastName'] = lastName2016
votersLast2018['lastName'] = lastName2018
#votersLast2014.iloc[:10]

In [81]:
matchingChairs

,Committee,C_First_Name,C_Last_Name,Chair_Names
0,"Agriculture, Nutrition and Forestry",Pat,Roberts,Pat Roberts
1,Appropriations,Richard,Shelby,Richard Shelby
2,Armed Services,Jim,Inhofe,Jim Inhofe
3,"Banking, Housing and Urban Affairs",Mike,Crapo,Mike Crapo
4,Budget,Mike,Enzi,Mike Enzi
5,"Commerce, Science and Transportation",Roger,Wicker,Roger Wicker
6,Energy and Natural Resources,Lisa,Murkowski,Lisa Murkowski
7,Environment and Public Works,John,Barrasso,John Barrasso
8,Finance,Chuck,Grassley,Chuck Grassley
9,Foreign Relations,Jim,Risch,Jim Risch


In [82]:
print(len(votersLast2016))

163


In [83]:
#Keep this code!!!!!
def matchLists(longerList, smallerList):
    index = []
    for i in smallerList:
        #print(i)
        for j in range(len(longerList)):
            if i in longerList[j]:
                #print(j)
                index.append(j)
    print(f'length with all: {len(index)}')
    index = sorted(index)
    #print(index)
    uniqueL = []
    for x in index:
        if x not in uniqueL:
            uniqueL.append(x)
    print(f'length uniques only: {len(uniqueL)}')
    return uniqueL

In [84]:
votersLast2018.head()

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,lastName
0,Republican,Martha McSally,McSally,Martha,AZ,1135200,Arizona,2384308,McSally
1,Democrat,Kyrsten Sinema,Sinema,Kyrsten,AZ,1191100,Arizona,2384308,Sinema
2,green,Angela Green,Green,Angela,AZ,57442,Arizona,2384308,Green
3,Democrat,Dianne Feinstein,Feinstein,Dianne,CA,6019422,California,11113364,Feinstein
4,Democrat,Kevin De Leon,Leon,Kevin,CA,5093942,California,11113364,Leon


In [85]:
searchListVoters2014 = votersLast2014['lastName']
searchListVoters2016 = votersLast2016['lastName']
searchListVoters2018 = votersLast2018['lastName']
searchListChairs = matchingChairs['C_Last_Name']

In [86]:
#searchListChairs

In [87]:
#Test if Last Name actually Matches
#Will throw error if names don't match
votersLast2018['lastName'] = lastName2018
#votersLast2018.iloc[:10]

In [88]:
def findString(searchString, searchList):
    i = 0
    #searchString = 'McSally'
    while i < len(searchList):
        if searchList[i] == searchString:
            print(i)
            i+= 1
        else:
            i += 1
    return i

# Consolidating Votes, Committees and Contributions 2014

## Adding Votes - Chairs 2014

In [89]:
#Looking for matches in Matching chairs via searchlistchairs
# for i in searchListChairs:
#     findString(i, searchListVoters2014)
# Manually Copied previous list for expediency
listNums2014 = [38, 95, 146, 27, 122, 62, 92, 109]
for i in listNums2014:
    print(votersLast2014['Last_Name'][i])

Roberts
Inhofe
Enzi
Risch
Alexander
Johnson
Johnson
Graham


In [90]:
matchingChairs.head()

,Committee,C_First_Name,C_Last_Name,Chair_Names
0,"Agriculture, Nutrition and Forestry",Pat,Roberts,Pat Roberts
1,Appropriations,Richard,Shelby,Richard Shelby
2,Armed Services,Jim,Inhofe,Jim Inhofe
3,"Banking, Housing and Urban Affairs",Mike,Crapo,Mike Crapo
4,Budget,Mike,Enzi,Mike Enzi


In [91]:
chairs2014 = votersLast2014.iloc[listNums2014, :]
chairs2014 = chairs2014.reset_index()
chairs2014 = chairs2014.drop(['index'], axis = 1)
#chairs2014

In [92]:
#matchingChairs

In [93]:
# for i in chairs2014['Last_Name']:
#     findString(i, matchingChairs["C_Last_Name"])

## Adding Committee - Chairs 2014

In [94]:
#Matching names to committees from the two lists
listNums2014_2 = [0, 2, 4, 9, 10, 11, 11, 12]
for i in listNums2014_2:
    print(f'Committee :{matchingChairs["Committee"][i]} Name: {matchingChairs["Chair_Names"][i]}')
#Dropping Heather and Connie Johnson, they are Johnsons but not committee members
committee2014 = matchingChairs.iloc[listNums2014_2, :]
chairs2014["Committee"] = list(committee2014['Committee'])
chairs2014 = chairs2014.drop([5,6])
chairs2014 = chairs2014.reset_index()
chairs2014 = chairs2014.drop(['index'], axis = 1)
chairs2014

Committee :Agriculture, Nutrition and Forestry Name: Pat Roberts
Committee :Armed Services Name: Jim Inhofe
Committee :Budget Name: Mike Enzi
Committee :Foreign Relations Name: Jim Risch
Committee :Health, Education, Labor and Pensions Name: Lamar Alexander
Committee :Homeland Security and Governmental Affairs Name: Ron Johnson
Committee :Homeland Security and Governmental Affairs Name: Ron Johnson
Committee :Judiciary Name: Lindsey Graham


,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,lastName,Committee
0,Republican,Pat Roberts,Roberts,Pat,KS,460350,Kansas,866191,Roberts,"Agriculture, Nutrition and Forestry"
1,Republican,James M. Inhofe,Inhofe,James,OK,558166,Oklahoma,820733,Inhofe,Armed Services
2,Republican,Michael B. Enzi,Enzi,Michael,WY,121554,Wyoming,171153,Enzi,Budget
3,Republican,James E. Risch,Risch,James,ID,285596,Idaho,437170,Risch,Foreign Relations
4,Republican,Lamar Alexander,Alexander,Lamar,TN,850087,Tennessee,1374065,Alexander,"Health, Education, Labor and Pensions"
5,Republican,Lindsey Graham,Graham,Lindsey,SC,672941,South Carolina,1240075,Graham,Judiciary


## Adding Contributions - Chairs 2014

In [95]:
contrib2014_file = "campaignContributions/contributions_2014.csv"
contrib2014 = pd.read_csv(contrib2014_file, encoding= 'unicode_escape')

In [96]:
for i in range(len(contrib2014['Last_Name'])):
    if contrib2014['Last_Name'][i] == 'Inhofe':
        print(i)

93


In [97]:
for i in chairs2014['Last_Name']:
    #print(f'Searching:{i}')
    findString(i, contrib2014["Last_Name"])

67
93
27
97
0
33


In [98]:
listNums2014_2 = [67, 93, 27, 97, 0, 33]
contrib2014 = contrib2014.iloc[listNums2014_2, :]
# for i in listNums2014_2:
#     print(chairs2014['Last_Name'][i])

In [99]:
contrib2014

,First_Name,Last_Name,State,Party_Name,Total_Raised,Full_Name,State_Abbrv,Party
67,Pat,Roberts,Kansas,Republican,8040544,Pat Roberts,KS,R
93,James M.,Inhofe,Oklahoma,Republican,4841285,James M. Inhofe,OK,R
27,Mike,Enzi,Wyoming,Republican,3772045,Mike Enzi,WY,R
97,James E.,Risch,Idaho,Republican,2902855,James E. Risch,ID,R
0,Lamar,Alexander,Tennessee,Republican,8201067,Lamar Alexander,TN,R
33,Lindsey,Graham,South Carolina,Republican,11056889,Lindsey Graham,SC,R


In [100]:
chairs2014['Contributions'] = list(contrib2014['Total_Raised'])
chairs2014 = chairs2014.drop(['lastName'], axis = 1)
chairs2014

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,Committee,Contributions
0,Republican,Pat Roberts,Roberts,Pat,KS,460350,Kansas,866191,"Agriculture, Nutrition and Forestry",8040544
1,Republican,James M. Inhofe,Inhofe,James,OK,558166,Oklahoma,820733,Armed Services,4841285
2,Republican,Michael B. Enzi,Enzi,Michael,WY,121554,Wyoming,171153,Budget,3772045
3,Republican,James E. Risch,Risch,James,ID,285596,Idaho,437170,Foreign Relations,2902855
4,Republican,Lamar Alexander,Alexander,Lamar,TN,850087,Tennessee,1374065,"Health, Education, Labor and Pensions",8201067
5,Republican,Lindsey Graham,Graham,Lindsey,SC,672941,South Carolina,1240075,Judiciary,11056889


# Consolidating Votes, Committee and Contributions  Ranking 2014

## Adding Votes - Ranking 2014

In [101]:
votersLast2014

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,lastName
0,Republican,Jeff Sessions,Sessions,Jeff,AL,795606,Alabama,818090,Sessions
1,Democrat,Mark Begich,Begich,Mark,AK,129431,Alaska,282400,Begich
2,Libertarian,Mark S. Fish,Fish,Mark,AK,10512,Alaska,282400,Fish
3,nonaffiliated,Ted Gianoutsos,Gianoutsos,Ted,AK,5636,Alaska,282400,Gianoutsos
4,Republican,Dan Sullivan,Sullivan,Dan,AK,135445,Alaska,282400,Sullivan
...,...,...,...,...,...,...,...,...,...
143,No Party,Over Vote,Vote,Over,WY,130,Wyoming,171153,Vote
144,Libertarian,Joseph S. Porambo,Porambo,Joseph,WY,3677,Wyoming,171153,Porambo
145,No Party,Blank Vote,Vote,Blank,WY,2633,Wyoming,171153,Vote
146,Republican,Michael B. Enzi,Enzi,Michael,WY,121554,Wyoming,171153,Enzi


In [102]:
#contrib2014.tail(50)

In [103]:
#matchingRanking

In [104]:
# #Looking for matches in Matching chairs via searchlistchairs
# for i in matchingRanking['R_Last_Name']:
#     findString(i, votersLast2014['lastName'])
#Manually Copied previous list for expediency
listNums2014_3 = [104,77,60]
ranking2014 = votersLast2014.iloc[listNums2014_3, :]
ranking2014 = ranking2014.drop([77])
ranking2014 = ranking2014.reset_index()
ranking2014 = ranking2014.drop('index', axis=1)
ranking2014

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,lastName
0,Democrat,Jack Reed,Reed,Jack,RI,223675,Rhode Island,316898,Reed
1,Democrat,Gary C. Peters,Peters,Gary,MI,1704936,Michigan,3121771,Peters


## Adding Committee - Ranking 2014

In [105]:
#Finding Committee
listNums2014_4 = [2, 11]
# for i in ranking2014['lastName']:
#     findString(i, matchingRanking['R_Last_Name'])
rankingCommittee2014 = [matchingRanking['Committee'][2], matchingRanking['Committee'][11]]
ranking2014['Committee'] = rankingCommittee2014

In [106]:
# Final Ranking 2014

# Adding Contributions - Ranking 2014

In [107]:
contrib2014_file = "campaignContributions/contributions_2014.csv"
contrib2014C = pd.read_csv(contrib2014_file, encoding= 'unicode_escape')

In [108]:
#matchingRanking

In [109]:
ranking2014

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,lastName,Committee
0,Democrat,Jack Reed,Reed,Jack,RI,223675,Rhode Island,316898,Reed,Armed Services
1,Democrat,Gary C. Peters,Peters,Gary,MI,1704936,Michigan,3121771,Peters,Homeland Security and Governmental Affairs


In [110]:
#contrib2014C Reed 3960335, Peters looked up on OpenSecrets

In [111]:
#https://www.opensecrets.org/members-of-congress/gary-peters/summary?cid=N00029277

In [112]:
ranking2014['Contributions'] = [3960335, 42543692]
ranking2014

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,lastName,Committee,Contributions
0,Democrat,Jack Reed,Reed,Jack,RI,223675,Rhode Island,316898,Reed,Armed Services,3960335
1,Democrat,Gary C. Peters,Peters,Gary,MI,1704936,Michigan,3121771,Peters,Homeland Security and Governmental Affairs,42543692


# Consolidating Votes, Committee & Chairs 2016

## Adding Votes - Chairs 2016

In [113]:
# for i in searchListChairs:
#     findString(i, searchListVoters2016)

In [114]:
listNums2016 = [1, 47, 7, 56, 161, 93, 36, 63]
# for i in listNums2016:
#     print(votersLast2016['lastName'][i])

In [115]:
chairs2016 = votersLast2016.iloc[listNums2016, :]
#chairs2016
chairs2016 = chairs2016.reset_index()
chairs2016 = chairs2016.drop(['index'], axis = 1)
# chairs2016['Committee'] = matchingChairs['Committee']
chairs2016

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,lastName
0,Republican,Richard C. Shelby,Shelby,Richard,AL,1335104,Alabama,2087444,Shelby
1,Republican,Mike Crapo,Crapo,Mike,ID,449017,Idaho,678943,Crapo
2,Republican,Lisa Murkowski,Murkowski,Lisa,AK,138149,Alaska,311441,Murkowski
3,Republican,Chuck Grassley,Grassley,Chuck,IA,926007,Iowa,1541036,Grassley
4,Libertarian,Philip N. Anderson,Anderson,Philip,WI,87531,Wisconsin,2948741,Anderson
5,Republican,Roy Blunt,Blunt,Roy,MO,1378458,Missouri,2802641,Blunt
6,Republican,Marco Rubio,Rubio,Marco,FL,4835191,Florida,9301820,Rubio
7,Republican,Jerry Moran,Moran,Jerry,KS,732376,Kansas,1177922,Moran


In [116]:
# for i in chairs2016['Last_Name']:
#     findString(i, matchingChairs["C_Last_Name"])

## Adding Committee - Chairs 2016

In [117]:
#Matching names to committees from the two lists
listNums2016_2 = [1, 3, 6, 8, 13, 14, 15]
for i in listNums2016_2:
    print(f'Committee :{matchingChairs["Committee"][i]} Name: {matchingChairs["Chair_Names"][i]}')
#Dropping Heather and Connie Johnson, they are Johnsons but not committee members
committee2016 = matchingChairs.iloc[listNums2016_2, :]

Committee :Appropriations Name: Richard Shelby
Committee :Banking, Housing and Urban Affairs Name: Mike Crapo
Committee :Energy and Natural Resources Name: Lisa Murkowski
Committee :Finance Name: Chuck Grassley
Committee :Rules and Administration Name: Roy Blunt
Committee :Small Business and Entrepreneurship Name: Marco Rubio
Committee :Veterans' Affairs Name: Jerry Moran


In [118]:
committee2016

,Committee,C_First_Name,C_Last_Name,Chair_Names
1,Appropriations,Richard,Shelby,Richard Shelby
3,"Banking, Housing and Urban Affairs",Mike,Crapo,Mike Crapo
6,Energy and Natural Resources,Lisa,Murkowski,Lisa Murkowski
8,Finance,Chuck,Grassley,Chuck Grassley
13,Rules and Administration,Roy,Blunt,Roy Blunt
14,Small Business and Entrepreneurship,Marco,Rubio,Marco Rubio
15,Veterans' Affairs,Jerry,Moran,Jerry Moran


In [119]:
chairs2016 = chairs2016.drop([4])

In [120]:
chairs2016["Committee"] = list(committee2016['Committee'])
chairs2016

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,lastName,Committee
0,Republican,Richard C. Shelby,Shelby,Richard,AL,1335104,Alabama,2087444,Shelby,Appropriations
1,Republican,Mike Crapo,Crapo,Mike,ID,449017,Idaho,678943,Crapo,"Banking, Housing and Urban Affairs"
2,Republican,Lisa Murkowski,Murkowski,Lisa,AK,138149,Alaska,311441,Murkowski,Energy and Natural Resources
3,Republican,Chuck Grassley,Grassley,Chuck,IA,926007,Iowa,1541036,Grassley,Finance
5,Republican,Roy Blunt,Blunt,Roy,MO,1378458,Missouri,2802641,Blunt,Rules and Administration
6,Republican,Marco Rubio,Rubio,Marco,FL,4835191,Florida,9301820,Rubio,Small Business and Entrepreneurship
7,Republican,Jerry Moran,Moran,Jerry,KS,732376,Kansas,1177922,Moran,Veterans' Affairs


## Adding Contributions - Chairs 2016

In [121]:
#matchingChairs

In [122]:
contrib2016_file = "campaignContributions/contributions_2016.csv"
contrib2016 = pd.read_csv(contrib2016_file, encoding= 'unicode_escape')

In [123]:
contrib2016

,First_Name,Last_Name,State,Party_Name,Total_Raised,Full_Name,State_Abbrv,Party
0,Lamar,Alexander,Tennessee,Republican,7976089,Lamar Alexander,TN,R
1,Kelly,Ayotte,New Hampshire,Republican,18428000,Kelly Ayotte,NH,R
2,Tammy,Baldwin,Wisconsin,Democrat,18657971,Tammy Baldwin,WI,D
3,John,Barrasso,Wyoming,Republican,5807201,John A Barrasso,WY,R
4,Michael,Bennet,Colorado,Democrat,17801104,Michael F Bennet,CO,D
...,...,...,...,...,...,...,...,...
95,Mark,Warner,Virginia,Democrat,16786930,Mark Warner,VA,D
96,Elizabeth,Warren,Massachusetts,Democrat,51431099,Elizabeth Warren,MA,D
97,Sheldon,Whitehouse,Rhode Island,Democrat,4528600,Sheldon Whitehouse,RI,D
98,Roger,Wicker,Mississippi,Republican,3832084,Roger Wicker,MS,R


In [124]:
for i in contrib2016['Last_Name']:
    if i == 'Grassley':
        print(i)

Grassley


In [125]:
#Looking for matches in Matching chairs via searchlistchairs
for i in chairs2016['lastName']:
    findString(i, contrib2016['Last_Name'])
#Manually Copied previous list for expediency

86
25
64
38
6
78
63


In [126]:
listNums2016_1 = [86,25,64,38,6,78,63]
addContrib = contrib2016.iloc[listNums2016_1, :]
addContrib
chairs2016['Contributions'] = list(addContrib['Total_Raised'])
chairs2016 = chairs2016.drop(['lastName'], axis = 1)

# Final Chairs 2016

In [127]:
chairs2016

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,Committee,Contributions
0,Republican,Richard C. Shelby,Shelby,Richard,AL,1335104,Alabama,2087444,Appropriations,4890271
1,Republican,Mike Crapo,Crapo,Mike,ID,449017,Idaho,678943,"Banking, Housing and Urban Affairs",5875081
2,Republican,Lisa Murkowski,Murkowski,Lisa,AK,138149,Alaska,311441,Energy and Natural Resources,6058418
3,Republican,Chuck Grassley,Grassley,Chuck,IA,926007,Iowa,1541036,Finance,9828566
5,Republican,Roy Blunt,Blunt,Roy,MO,1378458,Missouri,2802641,Rules and Administration,15893492
6,Republican,Marco Rubio,Rubio,Marco,FL,4835191,Florida,9301820,Small Business and Entrepreneurship,25254738
7,Republican,Jerry Moran,Moran,Jerry,KS,732376,Kansas,1177922,Veterans' Affairs,6617848


# Consolidating Votes, Committees, and Contributions Ranking 2016

In [128]:
votersLast2016

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,lastName
0,Democrat,Ron Crumpton,Crumpton,Ron,AL,748709,Alabama,2087444,Crumpton
1,Republican,Richard C. Shelby,Shelby,Richard,AL,1335104,Alabama,2087444,Shelby
2,Libertarian,Joe Miller,Miller,Joe,AK,90825,Alaska,311441,Miller
3,independent,Ted Gianoutsos,Gianoutsos,Ted,AK,1758,Alaska,311441,Gianoutsos
4,Democrat,Ray Metcalfe,Metcalfe,Ray,AK,36200,Alaska,311441,Metcalfe
...,...,...,...,...,...,...,...,...,...
158,Republican,Chris Vance,Vance,Chris,WA,1329338,Washington,3243317,Vance
159,Democrat,Patty Murray,Murray,Patty,WA,1913979,Washington,3243317,Murray
160,Democrat,Russ Feingold,Feingold,Russ,WI,1380335,Wisconsin,2948741,Feingold
161,Libertarian,Philip N. Anderson,Anderson,Philip,WI,87531,Wisconsin,2948741,Anderson


In [129]:
for i in matchingRanking['R_Last_Name']:
    findString(i, votersLast2016['lastName'])

157
131
159


In [130]:
listNums2016_2 = [157,131,159]
addVoters2016 = votersLast2016.iloc[listNums2016_2, :]
addVoters2016

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,lastName
157,Democrat,Patrick J. Leahy,Leahy,Patrick,VT,192243,Vermont,320467,Leahy
131,Democrat,Ron Wyden,Wyden,Ron,OR,1105119,Oregon,1952478,Wyden
159,Democrat,Patty Murray,Murray,Patty,WA,1913979,Washington,3243317,Murray


In [131]:
for i in addVoters2016['lastName']:
    findString(i, matchingRanking['R_Last_Name'])
listNums2016_3 = [1,8, 10]
addCommittee2016 = matchingRanking.iloc[listNums2016_3, :]
addCommittee2016
ranking2016 = addVoters2016
ranking2016['Committee'] = list(addCommittee2016['Committee'])
ranking2016 = ranking2016.reset_index()
ranking2016 = ranking2016.drop(['index', 'lastName'], axis = 1)
ranking2016

1
8
10


/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,Committee
0,Democrat,Patrick J. Leahy,Leahy,Patrick,VT,192243,Vermont,320467,Appropriations
1,Democrat,Ron Wyden,Wyden,Ron,OR,1105119,Oregon,1952478,Finance
2,Democrat,Patty Murray,Murray,Patty,WA,1913979,Washington,3243317,"Health, Education, Labor and Pensions"


In [132]:
contrib2016_file = "campaignContributions/contributions_2016.csv"
contrib2016B = pd.read_csv(contrib2016_file, encoding= 'unicode_escape')
contrib2016B

,First_Name,Last_Name,State,Party_Name,Total_Raised,Full_Name,State_Abbrv,Party
0,Lamar,Alexander,Tennessee,Republican,7976089,Lamar Alexander,TN,R
1,Kelly,Ayotte,New Hampshire,Republican,18428000,Kelly Ayotte,NH,R
2,Tammy,Baldwin,Wisconsin,Democrat,18657971,Tammy Baldwin,WI,D
3,John,Barrasso,Wyoming,Republican,5807201,John A Barrasso,WY,R
4,Michael,Bennet,Colorado,Democrat,17801104,Michael F Bennet,CO,D
...,...,...,...,...,...,...,...,...
95,Mark,Warner,Virginia,Democrat,16786930,Mark Warner,VA,D
96,Elizabeth,Warren,Massachusetts,Democrat,51431099,Elizabeth Warren,MA,D
97,Sheldon,Whitehouse,Rhode Island,Democrat,4528600,Sheldon Whitehouse,RI,D
98,Roger,Wicker,Mississippi,Republican,3832084,Roger Wicker,MS,R


In [133]:
for i in ranking2016['Last_Name']:
    findString(i, contrib2016B['Last_Name'])

53
99
66


In [134]:
listNums2016_2 = [53,99,66]
addContrib2016 = contrib2016B.iloc[listNums2016_2, :]
ranking2016["Contributions"] = list(addContrib2016['Total_Raised'])
ranking2016

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,Committee,Contributions
0,Democrat,Patrick J. Leahy,Leahy,Patrick,VT,192243,Vermont,320467,Appropriations,4770988
1,Democrat,Ron Wyden,Wyden,Ron,OR,1105119,Oregon,1952478,Finance,12628463
2,Democrat,Patty Murray,Murray,Patty,WA,1913979,Washington,3243317,"Health, Education, Labor and Pensions",13524088


# Consolidating Votes, Committee & Chairs 2018

In [135]:
# Getting Votes 2018 Chairs
# for i in matchingChairs['C_Last_Name']:
#     findString(i, votersLast2018['lastName'])
listNums2018_1 = [44,123,74]
chairs2018 = votersLast2018.iloc[listNums2018_1, :]
#chairs2018
#
# Getting Committees 2018 Chairs
# for i in chairs2018['lastName']:
#     findString(i, matchingChairs['C_Last_Name'])
#
listNums2018_2=[5,7,11]
addCommittee2018 = matchingChairs.iloc[listNums2018_2, :]
chairs2018["Committee"] = list(addCommittee2018['Committee'])
#chairs2018
#
# Bringing in Contributions 2018
contrib2018_file = "campaignContributions/contributions_2018.csv"
contrib2018 = pd.read_csv(contrib2018_file, encoding= 'unicode_escape')
#contrib2018
#
#Getting Contribution 2018 Chairs
# for i in chairs2018['lastName']:
#     findString(i, contrib2018['Last_Name'])
listNums2018_3=[97,2,48]
addContrib2018 = contrib2018.iloc[listNums2018_3, :]
chairs2018["Contributions"] = list(addContrib2018['Total_Raised'])
chairs2018 = chairs2018.reset_index()
chairs2018 = chairs2018.drop(['index', 'lastName'], axis = 1)

/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Final Chairs 2018

In [136]:
chairs2018

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,Committee,Contributions
0,Republican,Roger F. Wicker,Wicker,Roger,MS,547619,Mississippi,1843803,"Commerce, Science and Transportation",6568817
1,Republican,John Barrasso,Barrasso,John,WY,136210,Wyoming,203420,Environment and Public Works,7380997
2,Libertarian,Gary E Johnson,Johnson,Gary,NM,107201,New Mexico,697012,Homeland Security and Governmental Affairs,20248897


# Consolidating Votes, Committee & Ranking 2018

In [137]:
#matchingRanking

In [138]:
# Getting Votes 2018 Ranking
# for i in matchingRanking['R_Last_Name']:
#     findString(i, votersLast2018['lastName'])
listNums2018_4=[31,80,110,116,118,9,65,3,36,24,54]
ranking2018 = votersLast2018.iloc[listNums2018_4, :]
# ranking2018
# Getting Committee 2018 Ranking
# for i in ranking2018['lastName']:
#     findString(i, matchingRanking['R_Last_Name'])
listNums2018_5 = [0,3,4,5,6,7,9,12,13,14,15]
addCommitteeRanking2018 = matchingRanking.iloc[listNums2018_5, :]
addCommitteeRanking2018
ranking2018['Committee'] = list(addCommitteeRanking2018['Committee'])
# ranking2018
# Getting Contributions 2018 Ranking
# for i in ranking2018['lastName']:
#     findString(i, contrib2018['Last_Name'])
listNums2018_6 = [87,8,79,10,58,13,63,29,53,12,89]
addRankingContrib2018 = contrib2018.iloc[listNums2018_6, :]
ranking2018['Contributions'] = list(addRankingContrib2018['Total_Raised'])

/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Final Ranking 2018

In [139]:
ranking2018

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,lastName,Committee,Contributions
31,Democrat,Debbie Stabenow,Stabenow,Debbie,MI,2214478,Michigan,4237271,Stabenow,"Agriculture, Nutrition and Forestry",17570218
80,Democrat,Sherrod Brown,Brown,Sherrod,OH,2355923,Ohio,4410898,Brown,"Banking, Housing and Urban Affairs",28659987
110,independent,Bernie Sanders,Sanders,Bernie,VT,183529,Vermont,272624,Sanders,Budget,11795106
116,Democrat,Maria Cantwell,Cantwell,Maria,WA,1803364,Washington,3086168,Cantwell,"Commerce, Science and Transportation",11667705
118,Democrat,Joe Manchin,Manchin,Joe,WV,288808,West Virginia,582911,Manchin,Energy and Natural Resources,9375411
9,Democrat,Thomas R Carper,Carper,Thomas,DE,217385,Delaware,362592,Carper,Environment and Public Works,4262961
65,Democrat,Robert Menendez,Menendez,Robert,NJ,1711654,New Jersey,3169310,Menendez,Foreign Relations,12271086
3,Democrat,Dianne Feinstein,Feinstein,Dianne,CA,6019422,California,11113364,Feinstein,Judiciary,16092233
36,democratic-farmer-labor,Amy Klobuchar,Klobuchar,Amy,MN,1566174,Minnesota,5184235,Klobuchar,Rules and Administration,10681536
24,Democrat,Ben Cardin,Cardin,Ben,MD,1491614,Maryland,2299889,Cardin,Small Business and Entrepreneurship,5071964


# Merging Chairs

In [140]:
len(matchingChairs)

16

In [141]:
matchingChairs

,Committee,C_First_Name,C_Last_Name,Chair_Names
0,"Agriculture, Nutrition and Forestry",Pat,Roberts,Pat Roberts
1,Appropriations,Richard,Shelby,Richard Shelby
2,Armed Services,Jim,Inhofe,Jim Inhofe
3,"Banking, Housing and Urban Affairs",Mike,Crapo,Mike Crapo
4,Budget,Mike,Enzi,Mike Enzi
5,"Commerce, Science and Transportation",Roger,Wicker,Roger Wicker
6,Energy and Natural Resources,Lisa,Murkowski,Lisa Murkowski
7,Environment and Public Works,John,Barrasso,John Barrasso
8,Finance,Chuck,Grassley,Chuck Grassley
9,Foreign Relations,Jim,Risch,Jim Risch


In [142]:
add2014 = []
for i in range(len(chairs2014)):
    add2014.append('2014')
chairs2014['Election_Year']= add2014

In [143]:
len(chairs2014)

6

In [167]:
matchingChairs

,Committee,C_First_Name,C_Last_Name,Chair_Names
0,"Agriculture, Nutrition and Forestry",Pat,Roberts,Pat Roberts
1,Appropriations,Richard,Shelby,Richard Shelby
2,Armed Services,Jim,Inhofe,Jim Inhofe
3,"Banking, Housing and Urban Affairs",Mike,Crapo,Mike Crapo
4,Budget,Mike,Enzi,Mike Enzi
5,"Commerce, Science and Transportation",Roger,Wicker,Roger Wicker
6,Energy and Natural Resources,Lisa,Murkowski,Lisa Murkowski
7,Environment and Public Works,John,Barrasso,John Barrasso
8,Finance,Chuck,Grassley,Chuck Grassley
9,Foreign Relations,Jim,Risch,Jim Risch


In [144]:
chairs2014

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,Committee,Contributions,Election_Year
0,Republican,Pat Roberts,Roberts,Pat,KS,460350,Kansas,866191,"Agriculture, Nutrition and Forestry",8040544,2014
1,Republican,James M. Inhofe,Inhofe,James,OK,558166,Oklahoma,820733,Armed Services,4841285,2014
2,Republican,Michael B. Enzi,Enzi,Michael,WY,121554,Wyoming,171153,Budget,3772045,2014
3,Republican,James E. Risch,Risch,James,ID,285596,Idaho,437170,Foreign Relations,2902855,2014
4,Republican,Lamar Alexander,Alexander,Lamar,TN,850087,Tennessee,1374065,"Health, Education, Labor and Pensions",8201067,2014
5,Republican,Lindsey Graham,Graham,Lindsey,SC,672941,South Carolina,1240075,Judiciary,11056889,2014


In [145]:
add2016 = []
for i in range(len(chairs2016)):
    add2016.append('2016')
chairs2016['Election_Year']= add2016

In [146]:
len(chairs2016)

7

In [147]:
chairs2016

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,Committee,Contributions,Election_Year
0,Republican,Richard C. Shelby,Shelby,Richard,AL,1335104,Alabama,2087444,Appropriations,4890271,2016
1,Republican,Mike Crapo,Crapo,Mike,ID,449017,Idaho,678943,"Banking, Housing and Urban Affairs",5875081,2016
2,Republican,Lisa Murkowski,Murkowski,Lisa,AK,138149,Alaska,311441,Energy and Natural Resources,6058418,2016
3,Republican,Chuck Grassley,Grassley,Chuck,IA,926007,Iowa,1541036,Finance,9828566,2016
5,Republican,Roy Blunt,Blunt,Roy,MO,1378458,Missouri,2802641,Rules and Administration,15893492,2016
6,Republican,Marco Rubio,Rubio,Marco,FL,4835191,Florida,9301820,Small Business and Entrepreneurship,25254738,2016
7,Republican,Jerry Moran,Moran,Jerry,KS,732376,Kansas,1177922,Veterans' Affairs,6617848,2016


In [148]:
len(chairs2018)

3

In [149]:
add2018 = []
for i in range(len(chairs2018)):
    add2018.append('2018')
chairs2018['Election_Year']= add2018

In [150]:
chairs2018

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,Committee,Contributions,Election_Year
0,Republican,Roger F. Wicker,Wicker,Roger,MS,547619,Mississippi,1843803,"Commerce, Science and Transportation",6568817,2018
1,Republican,John Barrasso,Barrasso,John,WY,136210,Wyoming,203420,Environment and Public Works,7380997,2018
2,Libertarian,Gary E Johnson,Johnson,Gary,NM,107201,New Mexico,697012,Homeland Security and Governmental Affairs,20248897,2018


In [151]:
chairsAll = pd.concat([chairs2014, chairs2016, chairs2018])

In [152]:
len(chairsAll)

16

In [ ]:
chairsAll = chairsAll.reset_index()
chairsAll = chairsAll.drop('index', axis = 1)

In [168]:
#Ron Johnson is chair not gary johnson
chairsAll['Party'][15] = 'Republican'
chairsAll['Candidate'][15] = 'Ron Johnson'
chairsAll['Last_Name'][15] = 'Johnson'
chairsAll['First_Name'][15] = 'Ron'
chairsAll['State_Abbrv'][15] = 'WI'
chairsAll['Candidate_Votes'][15] = 1125999
chairsAll['State'][15] = 'Wisconsin'
chairsAll['Total_Votes'][15] = 2171331
chairsAll['Committee'][15] = "Homeland Security and Governmental Affairs"
chairsAll['Contributions'][15] =  18631193
chairsAll['Election_Year'][15] = 2014
chairsAll['Assumed_Office'][15] = 2011


/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [169]:
chairsAll['Assumed_Office'] = [1997, 1994, 1997, 2009, 2003, 2003, 1987, 1999, 2002, 1981, 2011, 2011, 2007, 2007, 2019, 2011]
chairsAll

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,Committee,Contributions,Election_Year,Assumed_Office
0,Republican,Pat Roberts,Roberts,Pat,KS,460350,Kansas,866191,"Agriculture, Nutrition and Forestry",8040544,2014,1997
1,Republican,James M. Inhofe,Inhofe,James,OK,558166,Oklahoma,820733,Armed Services,4841285,2014,1994
2,Republican,Michael B. Enzi,Enzi,Michael,WY,121554,Wyoming,171153,Budget,3772045,2014,1997
3,Republican,James E. Risch,Risch,James,ID,285596,Idaho,437170,Foreign Relations,2902855,2014,2009
4,Republican,Lamar Alexander,Alexander,Lamar,TN,850087,Tennessee,1374065,"Health, Education, Labor and Pensions",8201067,2014,2003
5,Republican,Lindsey Graham,Graham,Lindsey,SC,672941,South Carolina,1240075,Judiciary,11056889,2014,2003
6,Republican,Richard C. Shelby,Shelby,Richard,AL,1335104,Alabama,2087444,Appropriations,4890271,2016,1987
7,Republican,Mike Crapo,Crapo,Mike,ID,449017,Idaho,678943,"Banking, Housing and Urban Affairs",5875081,2016,1999
8,Republican,Lisa Murkowski,Murkowski,Lisa,AK,138149,Alaska,311441,Energy and Natural Resources,6058418,2016,2002
9,Republican,Chuck Grassley,Grassley,Chuck,IA,926007,Iowa,1541036,Finance,9828566,2016,1981


In [170]:
chairsAll.to_csv('Chairs.csv', index = False)

In [155]:
len(matchingRanking)

16

In [156]:
ranking2014['Election_Year'] = [2014, 2019]
ranking2014['Assumed_Office'] = [1997, 2019]

In [157]:
ranking2014

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,lastName,Committee,Contributions,Election_Year,Assumed_Office
0,Democrat,Jack Reed,Reed,Jack,RI,223675,Rhode Island,316898,Reed,Armed Services,3960335,2014,1997
1,Democrat,Gary C. Peters,Peters,Gary,MI,1704936,Michigan,3121771,Peters,Homeland Security and Governmental Affairs,42543692,2019,2019


In [158]:
add2016 = []
for i in range(len(ranking2016)):
    add2016.append('2016')
ranking2016['Election_Year']= add2016
ranking2016['Assumed_Office'] = [1975, 1996, 1993]

In [159]:
ranking2016

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,Committee,Contributions,Election_Year,Assumed_Office
0,Democrat,Patrick J. Leahy,Leahy,Patrick,VT,192243,Vermont,320467,Appropriations,4770988,2016,1975
1,Democrat,Ron Wyden,Wyden,Ron,OR,1105119,Oregon,1952478,Finance,12628463,2016,1996
2,Democrat,Patty Murray,Murray,Patty,WA,1913979,Washington,3243317,"Health, Education, Labor and Pensions",13524088,2016,1993


In [171]:
add2018 = []
for i in range(len(ranking2018)):
    add2018.append('2018')
ranking2018['Election_Year']= add2018
ranking2018['Assumed_Office'] =[2001, 2007, 2007, 2001, 2019, 2001, 2006, 1992, 2007, 2007, 2007]
ranking2018['Party'][8] = 'Democratic-Farmer-Labor'
#Wikipedia
#ranking2018
ranking2018 = ranking2018.reset_index()
ranking2018 = ranking2018.drop('index', axis=1)
ranking2018

/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,lastName,Committee,Contributions,Election_Year,Assumed_Office
0,Democrat,Debbie Stabenow,Stabenow,Debbie,MI,2214478,Michigan,4237271,Stabenow,"Agriculture, Nutrition and Forestry",17570218,2018,2001
1,Democrat,Sherrod Brown,Brown,Sherrod,OH,2355923,Ohio,4410898,Brown,"Banking, Housing and Urban Affairs",28659987,2018,2007
2,independent,Bernie Sanders,Sanders,Bernie,VT,183529,Vermont,272624,Sanders,Budget,11795106,2018,2007
3,Democrat,Maria Cantwell,Cantwell,Maria,WA,1803364,Washington,3086168,Cantwell,"Commerce, Science and Transportation",11667705,2018,2001
4,Democrat,Joe Manchin,Manchin,Joe,WV,288808,West Virginia,582911,Manchin,Energy and Natural Resources,9375411,2018,2019
5,Democrat,Thomas R Carper,Carper,Thomas,DE,217385,Delaware,362592,Carper,Environment and Public Works,4262961,2018,2001
6,Democrat,Robert Menendez,Menendez,Robert,NJ,1711654,New Jersey,3169310,Menendez,Foreign Relations,12271086,2018,2006
7,Democrat,Dianne Feinstein,Feinstein,Dianne,CA,6019422,California,11113364,Feinstein,Judiciary,16092233,2018,1992
8,Democratic-Farmer-Labor,Amy Klobuchar,Klobuchar,Amy,MN,1566174,Minnesota,5184235,Klobuchar,Rules and Administration,10681536,2018,2007
9,Democrat,Ben Cardin,Cardin,Ben,MD,1491614,Maryland,2299889,Cardin,Small Business and Entrepreneurship,5071964,2018,2007


In [172]:
rankingAll = pd.concat([ranking2014, ranking2016, ranking2018])
rankingAll = rankingAll.reset_index()
rankingAll = rankingAll.drop('index', axis = 1)
rankingAll

,Party,Candidate,Last_Name,First_Name,State_Abbrv,Candidate_Votes,State,Total_Votes,lastName,Committee,Contributions,Election_Year,Assumed_Office
0,Democrat,Jack Reed,Reed,Jack,RI,223675,Rhode Island,316898,Reed,Armed Services,3960335,2014,1997
1,Democrat,Gary C. Peters,Peters,Gary,MI,1704936,Michigan,3121771,Peters,Homeland Security and Governmental Affairs,42543692,2019,2019
2,Democrat,Patrick J. Leahy,Leahy,Patrick,VT,192243,Vermont,320467,NaN,Appropriations,4770988,2016,1975
3,Democrat,Ron Wyden,Wyden,Ron,OR,1105119,Oregon,1952478,NaN,Finance,12628463,2016,1996
4,Democrat,Patty Murray,Murray,Patty,WA,1913979,Washington,3243317,NaN,"Health, Education, Labor and Pensions",13524088,2016,1993
5,Democrat,Debbie Stabenow,Stabenow,Debbie,MI,2214478,Michigan,4237271,Stabenow,"Agriculture, Nutrition and Forestry",17570218,2018,2001
6,Democrat,Sherrod Brown,Brown,Sherrod,OH,2355923,Ohio,4410898,Brown,"Banking, Housing and Urban Affairs",28659987,2018,2007
7,independent,Bernie Sanders,Sanders,Bernie,VT,183529,Vermont,272624,Sanders,Budget,11795106,2018,2007
8,Democrat,Maria Cantwell,Cantwell,Maria,WA,1803364,Washington,3086168,Cantwell,"Commerce, Science and Transportation",11667705,2018,2001
9,Democrat,Joe Manchin,Manchin,Joe,WV,288808,West Virginia,582911,Manchin,Energy and Natural Resources,9375411,2018,2019


In [175]:
rankingAll.to_csv('consolidated_Ranking.csv', index = False)
chairsAll.to_csv('consolidate_Chairs.csv', index = False)